In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue May 15 18:49:42 2018

In [ ]:
@author: santanu
"""

In [ ]:
import shutil
import subprocess
import glob
from tqdm import tqdm
import numpy as np
import cv2
import os
from keras.applications.vgg16 import VGG16
from keras.models import Model 
import fire
from elapsedtimer import ElapsedTimer


<br>
VideoCaptioningPreProcessing will extract the Dense features from the <br>
images in the Video frame using a Pre-trained VGG16 CNN Model<br>


In [ ]:
class VideoCaptioningPreProcessing:
    
    def __init__(self,video_dest,feat_dir,
                 temp_dest,img_dim=224,channels=3,
                 batch_size=128,frames_step=80):
        
        self.img_dim = img_dim
        self.channels = channels
        self.video_dest = video_dest
        self.feat_dir = feat_dir
        self.temp_dest = temp_dest
        self.batch_cnn = batch_size
        self.frames_step = frames_step
        
        
        

Convert the video into image frames at a specified sampling rate 

In [ ]:
    def video_to_frames(self,video):
        
        with open(os.devnull, "w") as ffmpeg_log:
            if os.path.exists(self.temp_dest):
                print(" cleanup: " + self.temp_dest + "/")
                shutil.rmtree(self.temp_dest)
            os.makedirs(self.temp_dest)
            video_to_frames_cmd = ["ffmpeg",
                                       
                                       '-y',
                                       '-i', video,  
                                       '-vf', "scale=400:300", 
                                       '-qscale:v', "2", 
                                       '{0}/%06d.jpg'.format(self.temp_dest)]
            subprocess.call(video_to_frames_cmd,
                            stdout=ffmpeg_log, stderr=ffmpeg_log)
                        
# Load the pre-trained VGG16 Model and extract the dense features as output 
    def model_cnn_load(self):
         model = VGG16(weights = "imagenet", include_top=True,input_shape = (self.img_dim,self.img_dim,self.channels))
         out = model.layers[-2].output
         model_final = Model(inputs=model.input,outputs=out)
         return model_final

Load the video images 